# 导入模块

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
# import seaborn as sns 
#sns.set(style="darkgrid")
plt.rcParams["font.sans-serif"]=["SimHei"]
plt.rcParams["axes.unicode_minus"]=False


from dateutil.relativedelta import relativedelta
from datetime import datetime as dt
import time

import warnings  
warnings.filterwarnings("ignore")

import sys
import pymysql
from sqlalchemy import create_engine
import json
import datetime 
from datetime import timedelta ,time , datetime

# 连接数据库

In [2]:
def query(sql,
          host="rr-wz9wx0w3yti9d4f6wro.mysql.rds.aliyuncs.com",
          user="ylc",
          password="1O8t5lcJ5aMhwwPEUUjS",
          database = '' ,
          port=3306
         ):   
    conn = pymysql.connect(
        host=host,
        user=user,
        port=port,
        password=password,
        max_allowed_packet=1073741824,
        charset="utf8")
    try:
        df = pd.read_sql(sql, con=conn)
        conn.close()
    except:
        print('error')
        conn.close()
        raise
    return df

# 导入订单数据

In [3]:
sql1 = '''    
select o.order_number,od.product_id,od.product_name,o.merchant_id,o.merchant_name,o.`status`,o.create_time,op.pay_date,ol.go_express_date,o.alipay_order_id,cc.`name` as channel_name,pa.name as activity_name,o.order_method 
,tmer.shop_type
from db_digua_business.t_order o 
left join db_digua_business.t_order_pay op on op.order_id = o.id and op.pay_type = 'ZFBYSQ' and op.sync_mini_order = 'Y'
left join db_digua_business.t_order_logistics ol on o.id = ol.order_id 
left join db_digua_business.t_order_details od on od.order_id = o.id
-- 渠道名称
left join db_digua_business.t_channel cc on o.channel = cc.scene 
-- 活动名称
left join db_digua_business.t_platform_activity pa on o.activity_id = pa.id
LEFT JOIN db_digua_business.t_merchant tmer on tmer.id = o.merchant_id
where o.create_time >= DATE_ADD(CURRENT_DATE,INTERVAL -15 day ) 
and op.`status` in (2,5)
'''

df_order = query(sql1)
df_order.shape

(12588, 14)

In [4]:
df = df_order.copy()

# 处理日期

In [5]:
df["下单日期"]=pd.to_datetime(df["create_time"]).dt.date
df["下单日期"]=pd.to_datetime(df["下单日期"],errors="coerce")
df["下单日期"]
df["月份"]=df["下单日期"].dt.month
df.shape

(12588, 16)

# 渠道归属

In [6]:
df.loc[:,"来源渠道"]=df["channel_name"].fillna("未知渠道")

In [7]:
def qudao_type(a,b,c):
    a = str(a)
    b = str(b)
    if "租物" in b:
        return "芝麻租物"
    elif "芝麻" in b:
        return "芝麻租物"
    elif "抖音" in b:
        return "抖音渠道"
    elif "搜索" in a:
        return "搜索渠道"
    elif "租物" in a:
        return "芝麻租物"
    elif "芝麻" in a:
        return "芝麻租物"
    elif c == 1:
        return "芝麻租物"    
    elif "叮咚直播" in a:
        return "叮咚直播"
    elif "租瓜直播2号" in a:
        return "租瓜直播2号"
    elif "租瓜直播" in a:
        return "租瓜直播"
    elif "直播" in a:
        return "小舞直播"
    elif "繁星" in a:
        return "繁星"
    elif "生活号" in a:
        return "生活号"
    elif "群" in a:
        return "支付宝社群"
    elif "付费灯火" in a:
        return "付费灯火"
    else :
        return a
    

# %%
df.loc[:,"归属渠道"]=df.apply(lambda x:qudao_type(x["来源渠道"],x["activity_name"],x["order_method"]),axis=1)
df["归属渠道"].value_counts()

归属渠道
芝麻租物                         5312
搜索渠道                         5155
单人聊天会话中的小程序消息卡片（分享）           897
曙光计划                          315
九州信息                          179
我的小程序入口                       144
其他渠道场景渠道。                     112
生活号                           103
小舞直播                           82
付费流量(通过商家数字推广平台，灯火等投放的广告)      57
付费邦道-首页                        45
付费灯火                           44
付费侠客行-苹果旗舰                     39
支付宝 push 消息（同1014）             34
小程序商家消息（服务提醒）                  26
支付宝社群                          22
邦道月活BD02                        6
扫描二维码                           6
系统桌面图标                          2
支付宝客户端首页                        2
富士康租机                           1
线下易拉宝                           1
未知渠道                            1
邦道月活BD04                        1
收藏卡片01                          1
首页十二宫格及更多                       1
Name: count, dtype: int64

# 发货率

In [8]:
# def xcx(df):
#     df = df[df['发货时效']<=7]
#     df_group = df.groupby('订单生成时间').agg({'支付时间': 'count', '发货时间': 'count'})
#     # df_group.rename(columns={'支付时间': '进件量', '发货时间': '发货数'}, inplace=True)
#     df_group['发货率'] = df_group['发货时间'] / df_group['支付时间'] * 100
#     df_group['发货率'] = df_group['发货率'].apply(lambda x: f'{x: .2f}%')

#     return df_group

# 总体发货率

In [9]:
def xcx(df, date1, date2):
    df = df[(df['下单日期']>=date1)&(df['下单日期']<=date2)]
    df_all = df[['order_number', 'product_id', 'product_name', 'merchant_id',
       'merchant_name', 'status', 'create_time', 'pay_date', 'go_express_date',
       'alipay_order_id', 'channel_name',
       '下单日期', '月份', '来源渠道', '归属渠道']]
    df_all['create_time'] = df_all['create_time'].dt.date
    df_all['pay_date'] = df_all['pay_date'].dt.date
    df_all['go_express_date'] = df_all['go_express_date'].dt.date
    df_all.rename(columns={'order_number': '订单编号', 'product_id': '商品ID', 'product_name': '商品名称', 'merchant_id': '店铺ID', 'merchant_name': '店铺名称',
                        'status': '订单状态', 'create_time': '订单生成时间', 'pay_date': '支付时间', 'go_express_date': '发货时间', 'alipay_order_id': '支付宝流水号',
                        'order_number': '订单编号'}, inplace=True)
    df_all['发货时效'] = (pd.to_datetime(df_all['发货时间']) - pd.to_datetime(df_all['支付时间'])).dt.days
    df_all = df_all[['订单编号', '商品ID', '商品名称', '店铺ID',
        '店铺名称', '订单状态', '订单生成时间', '支付时间', '发货时间', '发货时效',
        '支付宝流水号', 'channel_name',
        '下单日期', '月份', '来源渠道', '归属渠道']]
    
    df_all_zh = df_all.groupby('订单生成时间').agg({'支付时间': 'count', '发货时间': 'count'})
    df_all_zh['发货率'] = df_all_zh['发货时间'] / df_all_zh['支付时间'] * 100
    df_all_zh['发货率'] = df_all_zh['发货率'].apply(lambda x: f'{x: .2f}%')
    df_all_zh.rename(columns={'支付时间': '进件量', '发货时间': '发货数'}, inplace=True)

    return df_all, df_all_zh


In [10]:
datetime.now()

datetime.datetime(2024, 12, 16, 11, 10, 1, 346573)

In [11]:
df_all, df_all_zh = xcx(df, '2024-10-01', datetime.now().strftime('%Y-%m-%d'))
df_all_zh

,进件量,发货数,发货率
订单生成时间,,,
2024-12-01,660,104,15.76%
2024-12-02,850,143,16.82%
2024-12-03,862,149,17.29%
2024-12-04,884,139,15.72%
2024-12-05,936,146,15.60%
2024-12-06,856,147,17.17%
2024-12-07,776,137,17.65%
2024-12-08,702,110,15.67%
2024-12-09,910,150,16.48%


In [12]:
df_2 = df[df['shop_type']!=2]
df_all_2, df_all_zh_2 = xcx(df_2, '2024-10-01', datetime.now().strftime('%Y-%m-%d'))
df_all_zh

,进件量,发货数,发货率
订单生成时间,,,
2024-12-01,660,104,15.76%
2024-12-02,850,143,16.82%
2024-12-03,862,149,17.29%
2024-12-04,884,139,15.72%
2024-12-05,936,146,15.60%
2024-12-06,856,147,17.17%
2024-12-07,776,137,17.65%
2024-12-08,702,110,15.67%
2024-12-09,910,150,16.48%


# 芝麻发货率

In [13]:
df_zms = df[df['归属渠道']=='芝麻租物']
df_zm, df_zm_zh = xcx(df_zms, '2024-10-01', datetime.now().strftime('%Y-%m-%d'))
df_zm_zh

,进件量,发货数,发货率
订单生成时间,,,
2024-12-01,305,46,15.08%
2024-12-02,334,58,17.37%
2024-12-03,345,45,13.04%
2024-12-04,348,37,10.63%
2024-12-05,394,65,16.50%
2024-12-06,380,63,16.58%
2024-12-07,341,55,16.13%
2024-12-08,294,42,14.29%
2024-12-09,364,68,18.68%


# 芝麻租物iPhone系列发货率

In [14]:
df_zm_iPhone = df_zms[df_zms['product_name'].str.contains(r'iPhone15|iPhone 15|iPhone16|iPhone 16')]
df_zm_iPhone, df_zm_iPhone_zh = xcx(df_zm_iPhone, '2024-10-01', datetime.now().strftime('%Y-%m-%d'))
df_zm_iPhone_zh

,进件量,发货数,发货率
订单生成时间,,,
2024-12-01,291,44,15.12%
2024-12-02,313,58,18.53%
2024-12-03,328,42,12.80%
2024-12-04,323,36,11.15%
2024-12-05,373,64,17.16%
2024-12-06,357,63,17.65%
2024-12-07,323,54,16.72%
2024-12-08,273,41,15.02%
2024-12-09,335,64,19.10%


# 定时任务

In [15]:
from apscheduler.schedulers.blocking import BlockingScheduler
from apscheduler.schedulers.background import BackgroundScheduler

hour = 18
minute = 29
# H = datetime.now().strftime('%H')
# M = datetime.now().strftime('%M')
def my_job1():
    print(f'执行定时任务：现在是每日的{hour}点{minute}分')
    # print(f'执行定时任务：现在是每日的{H}点{int(M)+1}分')
    path = r'\\digua\迪瓜租机\19.小程序发货率/'
    Today1 = str(datetime.now().strftime('%Y%m%d%H'))
    with pd.ExcelWriter(path + f'小程序发货率_{Today1}.xlsx', engine='openpyxl') as writer:
        df_all_zh.to_excel(writer, sheet_name='小程序发货率_全域转化')
    with pd.ExcelWriter(path + f'小程序发货率_{Today1}.xlsx', engine='openpyxl', mode='a') as writer:
        df_zm_zh.to_excel(writer, sheet_name='小程序发货率_芝麻转化')
        df_zm_iPhone_zh.to_excel(writer, sheet_name='小程序发货率_iPhone转化')
        df_all.to_excel(writer, sheet_name='小程序发货率_全域明细', index=False)
        df_all_2.to_excel(writer, sheet_name='小程序发货率_全域明细_自营', index=False)
        df_zm.to_excel(writer, sheet_name='小程序发货率_芝麻明细', index=False)
        df_zm_iPhone.to_excel(writer, sheet_name='小程序发货率_iPhone明细')
scheduler1 = BackgroundScheduler()
# 添加一个cron任务，每天的9:00执行
scheduler1.add_job(my_job1, 'cron', hour=hour, minute=minute)
# scheduler1.add_job(my_job1, 'cron', hour=H, minute=int(M)+1)

# 启动scheduler1
scheduler1.start()


执行定时任务：现在是每日的18点29分
执行定时任务：现在是每日的18点29分
执行定时任务：现在是每日的18点29分


In [16]:
# Today1 = str(datetime.now().strftime('%Y%m%d%H'))
